In [ ]:
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd
import os 
import os
import scanpy as sc
import numpy as np
import pandas as pd
import string
import anndata
from gtfparse import read_gtf
from anndata import AnnData
from collections import defaultdict

pd.set_option('display.max_rows', 500)

output = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/241010_make_raw_h5ad'
os.makedirs(output, exist_ok=True)

In [ ]:
adata_orig = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')


In [ ]:
adata_orig

In [ ]:
gtf_fn = '/media/carsten/hdd/genomes/human/cellranger/refdata-gex-GRCh38-2020-A/genes/genes.gtf'
data = '/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/cellranger_output'
def read_adata(folder):
    adata = sc.read_mtx(f'{folder}/outs/filtered_feature_bc_matrix/matrix.mtx.gz').T
    features = pd.read_csv(f'{folder}/outs/filtered_feature_bc_matrix/features.tsv.gz',
                          sep = '\t',
                          header = None)
    bc = pd.read_csv(f'{folder}/outs/filtered_feature_bc_matrix/barcodes.tsv.gz',
                          sep = '\t',
                    header = None)
    features.rename(columns={0:'gene_id',
                            1: 'gene_symbol',
                            2: 'category'},
                   inplace = True)

    adata.var = features
    adata.obs_names = bc[0]
    adata.var_names = adata.var['gene_id'].values
    return adata

In [ ]:
runs = os.listdir(data)
adatas = []
gtf = read_gtf(gtf_fn)
print(gtf.columns)
gtf['gene_id'] = gtf['gene_id'].str.split('.').str[0]

gene_name_dict = pd.Series(gtf.gene_name.values, index=gtf.gene_id).to_dict()
for x in gene_name_dict.keys():
    if gene_name_dict[x] == '':
        gene_name_dict[x] = x
for run in runs:
    print(run)
    folder = f'{data}/{run}'
    adata = read_adata(folder)
    adata.var_names = [gene_name_dict[x] for x in adata.var_names]
    adata.var_names_make_unique()
    adata.obs_names = run + '_' + adata.obs_names
    adata.obs['Patient'] = run.split('_')[-2]
    adata.obs['Group'] = run.split('_')[0]
    adata.obs['Contractility'] =[run.split('_')[1] if run.startswith('TN') else 'ND'][0]
    adata.obs['GroupContract'] = adata.obs['Group'].astype('str') + '-' + adata.obs['Contractility'].astype('str')
    adata.obs['Term'] = ['Term' if run.startswith('T') else 'Preterm'][0]
    adata.obs['Labor'] = ['Nonlaboring' if run[1]=='N' else 'Laboring'][0]
    adatas.append(adata.copy())
adata = anndata.concat(adatas)    


In [ ]:
for column in ['gene_id', 'gene_type', 'seqname', 'transcript_name', 'protein_id']:
    temp_dict = pd.Series(gtf[column].values, index=gtf['gene_name']).to_dict()
    temp_dict.update(pd.Series(gtf[column].values, index=gtf['gene_id']).to_dict())
    temp_dict = defaultdict(lambda: None, temp_dict)
    adata.var[column] = [temp_dict[x] for x in adata.var.index]

In [ ]:
adata1 = adata[adata_orig.obs_names,:].copy()
adata1.obs = adata_orig.obs.copy()
adata1.var['seqname'].unique()

In [ ]:
adata1

In [ ]:
adata1.write(f'{output}/uterus_processed_celltyped_raw.gz.h5ad',compression='gzip')


In [ ]:
adata1 = adata[adata_orig.obs_names,adata_orig.var_names]
adata1
adata_orig.layers['raw'] = adata1.X.copy()


In [ ]:
adata2 = adata_orig.copy()
adata2.X = adata_orig.layers['raw'].copy()
sc.pp.normalize_total(adata2)
sc.pp.log1p(adata2)
sc.pl.umap(adata2,color='OXTR')

In [ ]:
sc.pl.umap(adata2,color='ACTA2')
sc.pl.umap(adata2,color='Cell Subtype')

In [ ]:
adata_orig.write('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped_raw.gz.h5ad')


In [ ]:
adata_orig.X = adata_orig.layers['raw'].copy()
del adata_orig.layers
adata_orig

In [ ]:
adata_orig.write('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped_raw.gz.h5ad',compression='gzip')


In [ ]:
sc.pl.umap(adata,color='Cell Subtype')

In [ ]:
adata.obs['Cell Subtype'].value_counts(normalize=True)

In [ ]:
adata.obs['Cell Subtype'].unique()

In [ ]:
sc.pl.dotplot(adata,['ACTA2','OXTR','DPP6','NOTCH3','AVPR1A','RYR2','TRPV4','SULF1'],groupby='Cell Subtype')

In [ ]:
sc.pl.dotplot(adata,['ACTA2','OXTR','DPP6','NOTCH3','PDE3A','PDE4D'],groupby='Cell Subtype')

In [ ]:
genes=['AC079793.1',
'ADGRL3',
'ARHGAP15',
'DPP6',
'EDIL3',
'FHL5',
'GALNT17',
'HDAC9',
'KCNMA1',
'LINGO2',
'MYH11',
'NOTCH3',
'PARM1',
'PCA3',
'PDE3A',
'PDE4D',
'PRUNE2',
'SORBS2',
'SYTL2',
]
sc.pl.dotplot(adata,genes,groupby='Cell Subtype')

In [ ]:
adata

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Uterine smooth muscle'],['SULF1','GREM1'],groupby='GroupContract')

In [ ]:

adata.obs['Cell Subtype'].value_counts(normalize=True)*100

In [ ]:
sc.pl.umap(adata,color=['ACTA2','TAGLN','MYH11','OXTR','NOTCH3'])

In [ ]:
sc.pl.dotplot(adata,
               ['ACTA2','TAGLN','MYH11','OXTR','NOTCH3','TRPV4','TRPC4'],
             groupby='Cell Subtype')

In [ ]:
sc.pl.umap(adata,color='OXTR')

In [ ]:
lin = 'Immune'
lin_adata = adata[adata.obs['Lineage']=='Immune']
sc.pl.embedding(lin_adata,
               basis=f'X_umap_{lin}',
               color=f'leiden_{lin}',
               legend_loc='on data'
               )
sc.pl.embedding(lin_adata,
               basis=f'X_umap_{lin}',
               color='Cell Subtype',
               legend_loc='on data')

In [ ]:
sc.pl.embedding(lin_adata,
               basis=f'X_umap_{lin}',
               color='TPSB2',
               legend_loc='on data')

In [ ]:
sc.pl.dotplot(lin_adata,
               ['TPSB2'],
             groupby='Cell Subtype')

In [ ]:
adata.obs['Cell Subtype'].unique()

In [ ]:
gene_ls=['SOX9','PGR','ESR1','MMP7','MKI67','WNT7A','KRT5','LGR5','MUC12','CDC20B','DPP4','PAEP','CXCL14','HES6','FOXJ1','CCNO','IL6','VTCN1','MSLN','DST','SMAP1','LAMA4','WNT7A','KRT17','PAEP',
        'PLAU','IL32','TNF','WNT7A','KRT17','KRT5','LGR5','IHH','EMID1','EPCAM','GPX3'
        ]


lin = adata[adata.obs['Lineage']=='Endometrial']
sc.pl.dotplot(lin,gene_ls,groupby='leiden_Endometrial')

In [ ]:
gene_ls=['NTM',
         'CD4',
        ]


lin = adata[adata.obs['Lineage']=='Immune']
sc.pl.dotplot(lin,gene_ls,groupby='leiden_Immune')

In [ ]:
sc.pl.embedding(adata[adata.obs['Lineage']=='Endothelial'],
               basis='X_umap_Endothelial',
               color='Cell Subtype',
               legend_loc='on data')

In [ ]:
sc.pl.embedding(adata[adata.obs['Lineage']=='Endometrial'],
               basis='X_umap_Endometrial',
               color='Cell Subtype',
               legend_loc='on data')

In [ ]:
sc.pl.embedding(adata[adata.obs['Lineage']=='Mesenchymal'],
               basis='X_umap_Mesenchymal',
               color='Cell Subtype',
               legend_loc='on data')

In [ ]:
sc.pl.embedding(lin,
               basis='X_umap_Immune',
               color=gene_ls)

In [ ]:

sc.pl.DotPlot(adata,['ACTA2','ADGRL3','DPP6','CDH5'],standard_scale='var',groupby='Cell Subtype').style(cmap='Reds').legend(width=2).add_totals().show()

In [ ]:
genes = ['TRPV4','RYR2','OXTR','NOS1','NOS2','NOS3']
# sc.pl.umap(adata,color = genes)
sc.pl.DotPlot(adata,genes,standard_scale='var',groupby='Cell Subtype').style(cmap='Reds').legend(width=2).add_totals().show()

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Artery'], 
              ['ECHDC2','ID3','CXCL2','TSPAN18'], 
              use_raw=False,
              groupby='GroupContract')

In [ ]:
genes = ['ECHDC2','ID3','CXCL2','TSPAN18', 'CXCR2','ACKR3']
# sc.pl.umap(adata,color = genes)
sc.pl.DotPlot(adata,genes,standard_scale='var',groupby='Cell Subtype').style(cmap='Reds').legend(width=2).add_totals().show()

In [ ]:
sc.pl.DotPlot(adata,['PAEP','B2M','MGP','DEC1'],groupby='Cell Subtype').style(cmap='Reds').legend(width=2).add_totals().show()

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Uterine smooth muscle'], 
              ['PAEP','B2M','MGP','DEC1','DDX24','ITM2B'], 
              use_raw=False,
              groupby='GroupContract')

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Matrix fibroblast'], 
              ['PER3'], 
              use_raw=False,
              groupby='GroupContract')

In [ ]:
genes = ['PER3']
# sc.pl.umap(adata,color = genes)
sc.pl.DotPlot(adata,genes,standard_scale='var',groupby='Cell Subtype').style(cmap='Reds').legend(width=2).add_totals().show()

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Matrix fibroblast'], 
              ['B3GALT5'], 
              use_raw=False,
              groupby='GroupContract')

In [ ]:
adata.obs.groupby('GroupContract')['Cell Subtype'].value_counts(normalize=True)

In [ ]:
sc.pl.dotplot(adata[adata.obs['Cell Subtype']=='Artery'], 
              ['LSAMP','CXCL2','SLFN12L','TSPAN18'], 
              use_raw=False,
              groupby='GroupContract')

In [ ]:
adata